In [1]:
!pip install transformers torch datasets langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=b10701ad57bb545745f1ca7a8bf952f962cfaf9bfe6d4a519024aebf3f3fa72c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
import torch
print(torch.cuda.is_available())  # Kết quả True nếu PyTorch nhận diện được GPU

False


In [3]:
from datasets import load_dataset

ds = load_dataset("hchautran/vietnamese-wiki")

README.md:   0%|          | 0.00/439 [00:00<?, ?B/s]

(…)-00000-of-00003-7e649b6e52679f61.parquet:   0%|          | 0.00/269M [00:00<?, ?B/s]

(…)-00001-of-00003-9954675ffd3b42e6.parquet:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

(…)-00002-of-00003-f39518f3a4d486af.parquet:   0%|          | 0.00/268M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1273469 [00:00<?, ? examples/s]

In [4]:
import pandas as pd

# Chuyển đổi Dataset thành DataFrame
df = ds['train'].to_pandas()

df.head()

,id,title,clean_text
0,2,Trang Chính,Trang Chính templatestyles src Wiki2021 styles...
1,4,Internet Society,Internet Society Internet Society hay ISOC là ...
2,13,Tiếng Việt,Tiếng Việt Tiếng Việt cũng gọi là tiếng Việt N...
3,24,Ohio,Ohio Ohio viết tắt là OH viết tắt cũ là O là m...
4,26,California,California California phát âm như Ca li phót n...


In [5]:
# In số lượng hàng trong DataFrame
print("Số lượng hàng trong DataFrame sau khi loại các dòng có ngôn ngữ E:", df.shape[0])

# In kết quả
print(df.head())

Số lượng hàng trong DataFrame sau khi loại các dòng có ngôn ngữ E: 1273469
   id             title                                         clean_text
0   2       Trang Chính  Trang Chính templatestyles src Wiki2021 styles...
1   4  Internet Society  Internet Society Internet Society hay ISOC là ...
2  13        Tiếng Việt  Tiếng Việt Tiếng Việt cũng gọi là tiếng Việt N...
3  24              Ohio  Ohio Ohio viết tắt là OH viết tắt cũ là O là m...
4  26        California  California California phát âm như Ca li phót n...


In [6]:
import re

# Biểu thức chính quy để tìm ngày tháng (dd/mm/yyyy hoặc d/m/yyyy)
date_pattern = r'\b(\d{1,2}/\d{1,2}/\d{4}|[0-9]{1,2} tháng [0-9]{1,2})\b'

# Loại bỏ các hàng có ngày tháng trong cột 'title'
df = df[~df['title'].str.contains(date_pattern, regex=True)]

# In số lượng hàng trong DataFrame
print("Số lượng hàng trong DataFrame sau khi xóa ngày tháng:", df.shape[0])

<ipython-input-6-d756a3b8a258>:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['title'].str.contains(date_pattern, regex=True)]


Số lượng hàng trong DataFrame sau khi xóa ngày tháng: 1272995


In [7]:
import pandas as pd
from langdetect import detect

# Hàm kiểm tra ngôn ngữ
def detect_language(text):
    try:
        lang = detect(text)
        if lang == 'en':
            return 'English'
        elif lang == 'vi':
            return 'Vietnamese'
        else:
            return 'Other'
    except:
        return 'Error'

# Áp dụng hàm lên cột 'text'
df['language'] = df['title'].apply(detect_language)

print(df)


               id             title  \
0               2       Trang Chính   
1               4  Internet Society   
2              13        Tiếng Việt   
3              24              Ohio   
4              26        California   
...           ...               ...   
1273464  19734222       Sông Neckar   
1273465  19734225          Kia Soul   
1273466  19734226   Lưu vong (phim)   
1273467  19734232              VNOI   
1273468  19734238          Vòng hoa   

                                                clean_text    language  
0        Trang Chính templatestyles src Wiki2021 styles...  Vietnamese  
1        Internet Society Internet Society hay ISOC là ...       Other  
2        Tiếng Việt Tiếng Việt cũng gọi là tiếng Việt N...  Vietnamese  
3        Ohio Ohio viết tắt là OH viết tắt cũ là O là m...       Other  
4        California California phát âm như Ca li phót n...     English  
...                                                    ...         ...  
1273464  Sông Neckar

In [8]:
# Lọc các dòng có ngôn ngữ là tiếng Anh
df_vnese = df[df['language'] == 'Vietnamese']

In [9]:
# Áp dụng để lấy clean_text từ vị trí sau title
df_vnese['text'] = df_vnese.apply(lambda row: row['clean_text'][len(row['title']):], axis=1)

print("Sau khi xử lí clean_text:")
df_vnese.head()

Sau khi xử lí clean_text:


<ipython-input-9-ed8104a44c40>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vnese['text'] = df_vnese.apply(lambda row: row['clean_text'][len(row['title']):], axis=1)


,id,title,clean_text,language,text
0,2,Trang Chính,Trang Chính templatestyles src Wiki2021 styles...,Vietnamese,templatestyles src Wiki2021 styles css __NOED...
2,13,Tiếng Việt,Tiếng Việt Tiếng Việt cũng gọi là tiếng Việt N...,Vietnamese,Tiếng Việt cũng gọi là tiếng Việt Nam hay Việ...
5,33,Thụy Điển,Thụy Điển Thụy Điển tên chính thức là Vương qu...,Vietnamese,Thụy Điển tên chính thức là Vương quốc Thụy Đ...
6,39,Thành phố Hồ Chí Minh,Thành phố Hồ Chí Minh Thành phố Hồ Chí Minh cò...,Vietnamese,Thành phố Hồ Chí Minh còn gọi bằng tên cũ phổ...
7,44,Lào Cai,Lào Cai Lào Cai là một tỉnh vùng cao biên giới...,Vietnamese,Lào Cai là một tỉnh vùng cao biên giới thuộc ...


In [10]:
#Xóa cột 
df_vnese = df_vnese.drop(columns = ['clean_text','language'])

print("Sau khi xóa các cột:")
df_vnese.head()

Sau khi xóa các cột:


,id,title,text
0,2,Trang Chính,templatestyles src Wiki2021 styles css __NOED...
2,13,Tiếng Việt,Tiếng Việt cũng gọi là tiếng Việt Nam hay Việ...
5,33,Thụy Điển,Thụy Điển tên chính thức là Vương quốc Thụy Đ...
6,39,Thành phố Hồ Chí Minh,Thành phố Hồ Chí Minh còn gọi bằng tên cũ phổ...
7,44,Lào Cai,Lào Cai là một tỉnh vùng cao biên giới thuộc ...


In [11]:
import pickle

# Lưu DataFrame với cột embedding vào file bằng pickle
with open('./data_import/data.pkl', 'wb') as f:
    pickle.dump(df_vnese, f)